<a href="https://colab.research.google.com/github/cristobalperezp/CC6205-NLP/blob/main/Tarea_3_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 3 - Word Embeddings 📚**

**Integrantes:**

**Fecha límite de entrega 📆:** 16 de mayo.

**Tiempo estimado de dedicación:**


**Instrucciones:**
- El ejercicio consiste en:
    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases.
    - Implementar el método de la Word Context Matrix. 
    - Entrenar Word2Vec y FastText sobre un pequeño corpus.
    - Evaluar los embeddings obtenidos en una tarea de clasificación.
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo **Jupyter Notebook**.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación. 


**Referencias**

Vídeos: 

- [Linear Models](https://youtu.be/zhBxDsNLZEA)
- [Neural Networks](https://youtu.be/oHZHA8h2xN0)
- [Word Embeddings](https://youtu.be/wtwUsJMC9CA)

## **Preguntas teóricas 📕 (3 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Modelos Lineales (1.5 ptos)**

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. 

Especifique: representación de los documentos de entrada, parámetros del modelo, transformaciones necesarias para obtener la probabilidad de cada etiqueta y función de pérdida escogida. **(0.75 puntos)**

**Respuesta**: 

Del dataset dado, se obtiene una representación TF-IDF del mismo que corresponderá al input del modelo, denominado por $\vec{X}$, dónde en las columnas se encuentra el vocabulario del dataset, en las filas los documentos y en cada entrada el TF-IDF correspondiente.

Para la clasificación de los documentos se utilizará el modelo log-lineal multiclase, dónde el output viene dado por:
$$\vec{\hat{y} } = softmax( \vec{X}*W + \vec{b})$$
con $softmax(\vec{z})_{[i]} = \frac{e^{\vec{x}_{[i]}}}{ \sum_{j} e^{\vec{x}_{[j]}} }$

Así:
$$\vec{\hat{y}}_{[i]} = \frac{e^{(\vec{x}*W + \vec{b})_{[i]}}}{ \sum_{j} e^{(\vec{x}*W + \vec{b})_{[j]}} }$$

Con la transformación softmax se forza a que los valores de las componentes de $\vec{\hat{y}}$ sean positivas y sumen uno, siendo así la probabilidad de pertenencia a cada clase.

Los parámetros del modelo son $W$ y $\vec{b}$, y la función de pérdida corresponderá a la cross-entropy, dada por:

$$L_{cross-entropy}(\vec{\hat{y}},\vec{y}) = -\sum_{i} \vec{y}_{[i]}*log(\vec{\hat{y}}_{[i]})$$

Se utiliza esta función de pérdida, ya que e objetivo buscado es obtener una interpretación probabilística dede los scores de cada clase.

**Pregunta 2**: Explique cómo funciona el proceso de entrenamiento en este tipo de modelos y su evaluación. **(0.75 puntos)**

**Respuesta**:

Para el entrenamiento del modelo log-lineal se encontrar los parámetros $W$ y $\vec{b}$ que minimizan la pérdida, esto queda escrito como:
$$\hat{\Theta} = (W,\vec{b}) = argmin_{\Theta} \frac{1}{n} \sum_{i=1}^{n} L_{cross-entropy}(\vec{\hat{y}},\vec{y})$$

$$= argmin_{(W,\vec{b})} \frac{1}{n} \sum_{i=1}^{n} L_{cross-entropy}(softmax( \vec{X}*W + \vec{b}),\vec{y}) $$

En consecuencia de no tener una solución cerrada, se utiliza el algoritmo de descenso de gradiente estocástico, que consiste de:

* Inicializar los parámetros con valores aleatorios o cero.
* Por cada iteración se calcula la pérdida con los valores actuales de los parámetros.
* Luego se actualizan siguiendo la siguiente regla hasta su convergencia:
* $\Theta = \Theta - ŋ*\nabla_{\Theta}L_{cross-entropy}(\vec{\hat{y}},\vec{y})$

Los criterios de convergencia se definen antes, y se pueden utilizar: número de iteraciones determinado o el tamaño del paso dado entre iteraciones es menor a un umbral especificado.

Como se busca obtener un modelo capaz de generalizar, se utiliza un split de la data, generando sub sets para entrenamiento, validación y test, de esta forma, con el dataset de entrenamiento se entrena el modelo, con el de validación se obtienen los valores óptimos de los hiperparámetros, y en testing se pone a prueba la performance del modelo utilizando una métrica de desempeño ad-hoc al problema de clasificación como *accuracy, presicion o recall*, entre otras.

---
El valor de ŋ corresponde a un hiperparámetro del modelo.

### **Parte 2: Redes Neuronales (1.5 ptos)** 

Supongamos que tenemos la siguiente red neuronal.

![image.png](https://drive.google.com/uc?export=view&id=1nV1G0dOeVGPn40qGcGF9l_pVEFNtLU-w)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, defina la salida $\vec{\hat{y}}$ en función del vector $\vec{x}$, pesos $W^i$, bias $b^i$ y funciones $g,f,h$. 

Adicionalmente liste y explicite las dimensiones de cada matriz y vector involucrado en la red neuronal. **(0.75 Puntos)**

**Respuesta**: 

Formula:

Si $f,g,h$ son las funciones de activación desde la primera capa a la tercera, y utilizando la notación vista en clases:
$$\vec{\hat{y}} = NN_{MLP3}(\vec{x}) = (h(g(f(\vec{x}W^1 + \vec{b^1})W^2 + \vec{b^2}))W^3 + \vec{b^3})W^4$$

Dimensiones: 

Como $\vec{x}$ está en $\Re^3$:

$dim(W^1)={3x2}$ y $dim(\vec{b^1})={2}$

$dim(W^2)={2x3}$ y $dim(\vec{b^2})={3}$

$dim(W^3)={3x1}$ y $dim(\vec{b^1})=1$

$dim(W^4)={1x4}$

**Pregunta 2**: Explique qué es backpropagation. ¿Cuales serían los parámetros a evaluar en la red neuronal anterior durante backpropagation? **(0.25 puntos)**

**Respuesta**:

Es un método para evaluar el gradiente de una función de pérdida $L$, para una red neuronal feed-forward, con respecto a todos sus parámetros, que utiliza la regla de la cadena y programación dinámica.

Los parámetros a evaluar en la raed anterior durante backpropagation, son $W^1,W^2,W^3,W^4$ y $\vec{b^1},\vec{b^2},\vec{b^3}$ .

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? **(0.5 puntos)**

**Respuesta**:

El algoritmo puede ser resumido en los siguientes pasos:
1.  Aplicar un vector de entrada $\vec{x}$ a la red y propagar hacia adelante a través de la red utilizando (eq.1)* y (eq.2) para encontrar las activaciones de todas las unidades ocultas y de salida.
$$eq.1: \vec{h_{[j]}^{l}}=(\sum_{i} W_{[i,j]}^{l}\ x\ \vec{z_{[i]}^{l-1}})+\vec{b_{j}^{l}}$$
$$eq.2:  \vec{z_{[j]}^{l}}=g(\vec{h_{[j]}^{l}})$$ con $\vec{z_{[j]}^{0}} = \vec{x_{[j]}}$

2. Evaluar $\vec{\delta_{[j]}^{m}}$ para todas las unidades de salida.

3. Hacer backpropagation en $\vec{\delta_{[k]}^{l+1}}$ utilizando (eq.3) para obtener $\vec{\delta_{[j]}^{l}}$ para cada capa oculta de la red. Pasando de las capas superiores a las inferiores de la red**.
$$eq.3: \vec{\delta_{[j]}^{l}}=g^{'}(\vec{h_{[j]}^{l}})\ x\ \sum_{k}(\vec{\delta_{[k]}^{l+1}}\ x\  W_{[j,k]}^{l+1})$$

4. Evaluar las derivadas requeridas utilizando $\frac{\delta L}{\delta W_{[i,j]}^{l}}=\vec{\delta_{[j]}^{l}}\ x\ \vec{z_{[i]}^{l-1}}$


---
*Los sesgos $\vec{b}$ pueden ser excluidos de (eq.1), incluyéndolos en las matrices $W$ introduciendo una unidad adicional, o entrada, cuya activación se fija en +1.

**Aquí vemos las derivadas que hay que computar para obtener $\vec{\delta_{[j]}^{l}}$ en cada capa, es decir las derivadas de las funciones $f,g,h$ ($g$ en la ecuación son las activaciones).

Así:
$$\vec{\delta^4} = \frac{\delta L}{\delta \vec{\hat{y}}}$$
$$\vec{\delta^3} = h^{'}(\vec{h^3})\vec{\delta^4}W^4 $$ 
$$\vec{\delta^2} = g^{'}(\vec{h^2})\vec{\delta^3}W^3$$
$$\vec{\delta^1} = f^{'}(\vec{h^1})\vec{\delta^2}W^2$$


## **Preguntas prácticas 💻 (3 puntos).** ##

### Parte 3 A (1 Punto): Word Contex Matrix



En esta parte debe crear una matriz palabra contexto, para esto, complete el siguiente template (para esta parte puede utilizar las librerías ```numpy``` y/o ```scipy```). Hint: revise como utilizar matrices sparse de ```scipy```

```python
class WordContextMatrix:

  def __init__(self, vocab_size, window_size, dataset, tokenizer):
    """
    Utilice el constructor para definir los parametros.
    """

    # se sugiere agregar un una estructura de datos para guardar las
    # palabras del vocab y para guardar el conteo de coocurrencia
    # si lo necesita puede agregar más parametros pero no puede cambiar el resto
    ...
    
  def build_vocab(self, word):
    """
    Utilice este método para construir el vocabulario
    """
    

    # Le puede ser útil considerar un token unk al vocab
    # para palabras fuera del vocab
    ...
  
  def build_matrix(self):
    """
    Utilice este método para crear la palabra contexto
    """
    ...

  def get_matrix(self):
    """
    Utilice este método para obtener la matriz palabra contexto. 
    """

    # se recomienda transformar la matrix a un diccionario de embedding.
    # por ejemplo {palabra1:vec1, palabra2:vec2, ...}
    ...

```

puede modificar los parámetros o métodos si lo considera necesario. Para probar la matrix puede utilizar el siguiente corpus.

```python
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]
```

Obteniendo una matriz parecia a esta:

***Resultado esperado***: 

| counts   | I  | like | enjoy | deep | learning | NLP | flying | . |   
|----------|---:|-----:|------:|-----:|---------:|----:|-------:|--:|
| I        | 0  |  2   |  1    |    0 |  0       |   0 | 0      | 0|            
| like     |  2 |    0 |  0    |    1 |  0       |   1 | 0      | 0 | 
| enjoy    |  1 |    0 |  0    |    0 |  0       |   0 | 1      | 0 |
| deep     |  0 |    1 |  0    |    0 |  1       |   0 | 0      | 0 |  
| learning |  0 |    0 |  0    |    1 |  0       |   0 | 0      | 1 |          
| NLP      |  0 |    1 |  0    |    0 |  0       |   0 | 0      | 1 |
| flying   |  0 |    0 |  1    |    0 |  0       |   0 | 0      | 1 | 
| .        |  0 |    0 |  0    |    0 |  1       |   1 | 1      | 0 | 

``

Verifique si su matrix es igual a esta utilizando el corpus de ejemplo. Ojo que este es sólo un ejemplo, su algoritmo debe **generalizar** a otros ejemplos.

### **Parte 3 B (1 Punto): Word Embeddings**

En la auxiliar 2 aprendieron como entrenar Word2Vec utilizando gensim. El objetivo de esta parte es comparar los embeddings obtenidos con dos modelos diferentes: Word2Vec y [FastText](https://radimrehurek.com/gensim/models/fasttext.html) (utilizen size=200 en FastText) entrenados en el mismo dataset de diálogos de los Simpson. 

In [ ]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Utilizando el dataset adjunto con la tarea:

In [ ]:
! git clone 'https://github.com/cristobalperezp/CC6205-NLP.git'

Cloning into 'CC6205-NLP'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 72 (delta 31), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (72/72), 6.68 MiB | 5.11 MiB/s, done.


In [ ]:
data_file = "/content/CC6205-NLP/dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec y FastText sobre el dataset anterior. **(1 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

**Pregunta 2**: Encuentre las palabras mas similares a las siguientes: Lisa, Bart, Homer, Marge. Cúal es la diferencia entre ambos resultados? Por qué ocurre esto? Intente comparar ahora Liisa en ambos modelos (doble i). Cuando escogería uno vs el otro? **(0.5 puntos)**

**Respuesta**:

### **Parte 4 (1 Punto): Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [ ]:
AFINN = 'AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos). 

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1). 

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.75 puntos)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

**Respuesta**:

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? **(0.75 puntos)**

**Respuesta**:

# Bonus: +0.25 puntos en cualquier pregunta

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**:

In [ ]:
AFINN = '/content/CC6205-NLP/AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

In [ ]:
df_afinn.head(10)

,0,1
0,tops,1
1,groan,-1
2,perfects,1
3,spammer,-1
4,saluting,1
5,transgresses,-1
6,punishes,-1
7,wasting,-1
8,virulent,-1
9,complaining,-1


In [ ]:
import gensim.downloader as api
modelo = api.load("glove-twitter-25") 

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

In [ ]:
def obtener_embedding(palabra):
    if try_apply(modelo, palabra):
        return modelo[palabra]
    else:
        return np.zeros(modelo.vector_size)

In [ ]:
df_afinn['embedding'] = df_afinn.loc[:,0].apply(obtener_embedding)

In [ ]:
df_afinn

,0,1,embedding
0,tops,1,"[0.49105, -0.87081, 0.49447, -1.1929, 0.29789,..."
1,groan,-1,"[-0.76202, 0.175, -0.28268, 1.2073, -0.25543, ..."
2,perfects,1,"[-0.036265, -0.22563, 0.54104, 0.00027051, -0...."
3,spammer,-1,"[0.03925, 0.93774, -0.71203, 0.23106, 0.20442,..."
4,saluting,1,"[-1.6916, 0.9639, -0.05709, 0.73704, -0.29311,..."
...,...,...,...
3377,mediocrity,-1,"[-0.62844, 0.34362, -2.1548, 1.0754, -0.64261,..."
3378,bold,1,"[0.12088, 0.13523, -0.37796, -0.16371, 0.84471..."
3379,hating,-1,"[-0.011326, 1.4475, 0.24511, 0.13498, -0.18077..."
3380,unfavorable,-1,"[0.53319, 0.19846, -0.7779, 0.13871, 0.42557, ..."


In [ ]:
df = df_afinn.copy()

In [ ]:
X = np.stack(df['embedding'])
y = df[1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logit = LogisticRegression(max_iter=1000000).fit(X_train, y_train)

In [ ]:
print(classification_report(y_train, logit.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.85      0.93      0.89      1985
           1       0.84      0.70      0.77      1058

    accuracy                           0.85      3043
   macro avg       0.85      0.82      0.83      3043
weighted avg       0.85      0.85      0.85      3043



In [ ]:
print(classification_report(y_test, logit.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.87      0.95      0.90       221
           1       0.88      0.73      0.80       118

    accuracy                           0.87       339
   macro avg       0.87      0.84      0.85       339
weighted avg       0.87      0.87      0.87       339



In [ ]:
conf_matrix = confusion_matrix(y_train, logit.predict(X_train))
print(conf_matrix)

[[1846  139]
 [ 316  742]]


In [ ]:
conf_matrix = confusion_matrix(y_test, logit.predict(X_test))
print(conf_matrix)

[[209  12]
 [ 32  86]]
